# Part 6: Verify

## Load the previous session

Generate the proof 

In [1]:
import copy
import math
import merkle
from field import FieldElement
from polynomial import interpolate_poly
from tutorial_sessions import part4
from channel import *

channel = part4()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1023/1023 [00:06<00:00, 151.18it/s]


## Verify the proof

Define the proof parameters (these should be encoded along with the proof)

In [2]:
cp_degree = 1023
group_size = 1024
blowup_factor = 8
num_rounds = math.ceil(math.log2(cp_degree))
num_random_queries = 3 

Reconstruct the composition polynomial

In [3]:
g = FieldElement.generator()**(3 * 2**20)
def p0(x, f):
    return (f - 1) / (x - 1)
def p1(x, f):
    return (f - 2338775057) / (x - g**1022)
def p2(x, f0, f1, f2):
    return (f2 - f1**2 - f0**2) / ((x**1024 - 1) / ((x - g**1021) * (x - g**1022) * (x - g**1023)))
def cp_0(x, f0, f1, f2):
    return alpha_0*p0(x, f0) + alpha_1*p1(x, f0) + alpha_2*p2(x, f0, f1, f2)

Generate the evaluation domain

In [4]:
h_gen = FieldElement.generator() ** ((2 ** 30 * 3) // 8192)
h = [h_gen ** i for i in range(8192)]
eval_domain = [FieldElement.generator() * x for x in h]

Make a copy of the proof transcript

In [5]:
proof = copy.copy(channel.proof)
proof.reverse()

Deserialize the following:
1. Merkle root of the trace polynomial evaluated over the evaluation domain
2. Random field elements supplied by the verifier used to construct the composition polynomial
3. Merkle root of the composition polynomial

In [6]:
f_merkle = decode_str(proof.pop())
alpha_0, alpha_1, alpha_2 = [decode_rand_field_elem(proof.pop()) for _ in range(3)]
cp_merkle = decode_str(proof.pop())

Verify {alpha_0, alpha_1, alpha_2} were sampled correctly using reconstructed channel state

In [7]:
# TODO

Deserialize FRI merkle tree roots and $\beta_i$ for each round $i$, including the final scalar codeword

In [8]:
betas = []
fri_merkles = []
for i in range(num_rounds):
    betas.append(decode_rand_field_elem(proof.pop()))
    fri_merkles.append(decode_str(proof.pop()))
fri_constant = decode_int(proof.pop())

Check that the last codeword matches the Merkle root

In [9]:
# Not needed because the final codeword is a scalar

Check the following:
- Final codeword is low degree
- Order of final evaluation domain matches final codeword length
- Final codeword remains the same when evaluated on the final evaluation domain

In [10]:
# No checks are needed because we did not terminate in an early round

Deserialize FRI layers

In [11]:
for i in range(num_random_queries):
    # TODO: Verify randomly sampled indices using reconstructed channel state
    idx = decode_rand_int(proof.pop())
    
    # Retrieve trace polynomial evaluations
    f_x = decode_field_elem(proof.pop())
    f_x_auth = decode_list(proof.pop())
    f_gx = decode_field_elem(proof.pop())
    f_gx_auth = decode_list(proof.pop())
    f_gsquaredx = decode_field_elem(proof.pop())
    f_gsquaredx_auth = decode_list(proof.pop())
    
    # Verify authentication paths for trace polynomial
    assert merkle.verify_decommitment(idx, f_x, f_x_auth, f_merkle) == True
    assert merkle.verify_decommitment(idx + 8, f_gx, f_gx_auth, f_merkle) == True
    assert merkle.verify_decommitment(idx + 16, f_gsquaredx, f_gsquaredx_auth, f_merkle) == True
    
    # Retrieve FRI decommitments
    layer_a = []
    layer_b = []
    auth_a = []
    auth_b = []
    idx_ = idx
    for _ in range(num_rounds):
        # Layer and auth path for idx
        layer_a.append(decode_field_elem(proof.pop()))
        auth_a.append(decode_list(proof.pop()))
        # Layer and auth path for sib_idx
        layer_b.append(decode_field_elem(proof.pop()))
        auth_b.append(decode_list(proof.pop()))
    # Final scalar codeword
    layer_a.append(decode_field_elem(proof.pop()))
        
    # Verify cp_0 is derived from the trace polynomial
    assert cp_0(eval_domain[idx], f_x, f_gx, f_gsquaredx) == layer_a[0]
        
    # Verify that FRI layers are consistent
    length = group_size * blowup_factor
    roots = [cp_merkle] + fri_merkles
    for n, (cp_i, cp_i_inv, cp_j) in enumerate(zip(layer_a[:-1], layer_b, layer_a[1:])):
        idx = int(idx % length)
        idx_sib = int((idx + length // 2) % length)
        
        # Verify authentication paths for composition polynomial
        assert merkle.verify_decommitment(idx_, layer_a[n], auth_a[n], roots[n]) == True
        assert merkle.verify_decommitment(idx_sib, layer_b[n], auth_b[n], roots[n]) == True
        
        # Verify colinearity 
        domain = [eval_domain[idx]**(2**n), eval_domain[idx_sib]**(2**n), betas[n]]
        values = [cp_i, cp_i_inv, cp_j]
        assert interpolate_poly(domain, values, disable_pbar=True).degree() == 1
        length /= 2
        
print("Verified")

Verified
